In [69]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

ProgrammingError: Oracle Client library has already been initialized

In [70]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

#crdmprd
conn2 = cx_Oracle.connect(
    user="AD21236",
    password="C#ntury123",
    dsn="racorap33-scan.corp.intranet/dwpr01p_users",
    encoding="UTF-8")

In [71]:
%%time

query2 = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,O.STATE
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  WIRE_CENTER_ID
                    ,STATE
                    ,count(distinct LUID) as cnt
                    
            FROM    QD
            
            GROUP BY
                    WIRE_CENTER_ID
                    ,STATE


"""

query = """

select distinct DTN, WIRE_CENTER_ID
from LQSMO.LU_CIRCUITS_O

"""


df_ora = pd.read_sql(query, con=conn) #look up for wc to dtn
df_ora2 = pd.read_sql(query2, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 453 ms
Wall time: 5min 56s


In [72]:
%%time

mg_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in (
                                          '1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306','1180028', '1178694', '1179569',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237', -- CRYSMNCR
                                          '1180315', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715' -- TCSNAZMA
                                            ) --add in MG polygon IDs here
                    
            )
            
            SELECT  WIRE_CENTER_ID
                    ,count(distinct LUID) as cnt
                    
            FROM    QD
            
            GROUP BY
                    WIRE_CENTER_ID


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 0 ns
Wall time: 1.34 s


In [73]:
mg_lu_df = mg_lu_ora.rename(columns = {'CNT': 'MG_ENABLED_LU_CNT'})

In [100]:
# Input excel MG raw data
sf_df = pd.read_excel(r'Salesforce Data/salesdata.xlsx',sheet_name = 'Ella XGS Sales') 

In [101]:
sf_df['Purchased Data Rate'].fillna('missing_speed', inplace=True)

In [102]:
# before import salesforce DTN field needs to be string
sf_df['DTN'] = sf_df['DTN'].apply(lambda x: '{:.0f}'.format(x))
sf_df['DTN'] = sf_df['DTN'].astype(str)

In [103]:
# select only account status Active or Pending Activation
sf_df = sf_df[(sf_df['Account Status'] == 'Active')|(sf_df['Account Status'] == 'Pending Activation')]

In [104]:
df_ora2.head()

,WIRE_CENTER_ID,STATE,CNT
0,HLDYUTMA,UT,265
1,NFMYFLXB,FL,176
2,MPLSMNTF,MN,929
3,PHNXAZSO,AZ,1155
4,PNISFLXA,FL,9073


In [105]:
# this has state/wirecenter/Enabled LU counts at wire center level
df_wc_state = df_ora2.copy()

In [106]:
#Adding wire center detail to Salesforce
combine_df = df_ora.merge(sf_df, how = 'outer', on = 'DTN') #combine now has wire center id
combine_df.head()

,DTN,WIRE_CENTER_ID,Wire Center,Completion Date,Account Name,Service Address,Service State Short,Polygon ID,Purchased Data Rate,Created Date,Account Status,Order Number,Work Order Number,Status,Appointment Number,Due Date,Created By: Full Name
0,1000357386,WNGRFLXA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1500018302,WNGRFLXA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000186631,PHNXAZMY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1500057070,PHNXAZMY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000200986,PHNXAZMY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
#Adding LU count detail to Sales force combine_df
combine_df_details = df_wc_state.merge(combine_df, how = 'outer', on = 'WIRE_CENTER_ID')
combine_df_details['STATE'].value_counts() #check for nulls

FL    21121
CO     2190
AZ     1666
UT     1457
MN     1367
WA      522
OR        6
Name: STATE, dtype: int64

In [108]:
#Update CNT column name
combine_df_details = combine_df_details.rename(columns = {'CNT': 'Enabled_LUs_WireCenter_Level'})

In [109]:
summary_df = combine_df_details.copy()
summary_df = summary_df.fillna(0)

In [110]:
summary_df['Order Number'] = np.where(summary_df['Order Number'] == 0.0, np.NaN, summary_df['Order Number'])

In [111]:
summary_df.head()

,WIRE_CENTER_ID,STATE,Enabled_LUs_WireCenter_Level,DTN,Wire Center,Completion Date,Account Name,Service Address,Service State Short,Polygon ID,Purchased Data Rate,Created Date,Account Status,Order Number,Work Order Number,Status,Appointment Number,Due Date,Created By: Full Name
0,HLDYUTMA,UT,265.0,1234567890,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0
1,HLDYUTMA,UT,265.0,1500063046,HLDYUTMA,10/27/2022 10:22 AM,MAREK KOBLANSKI,1758 E 4620 S SALT LAKE CITY Utah United State...,UT,1179968.0,940M/940M,10/25/2022,Active,A00193426,826476.0,Completed,SA-2259628,10/27/2022 12:40 PM,Px ServiceUser
2,HLDYUTMA,UT,265.0,1000334056,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0
3,HLDYUTMA,UT,265.0,1500068791,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0
4,HLDYUTMA,UT,265.0,1500060390,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0


In [113]:
# check if new speeds are popping up
summary_df['Purchased Data Rate'].value_counts()

0                135864
940M/940M           703
200M/200M           261
1G/1G                80
3G/3G                 5
missing_speed         1
8G/8G                 1
Name: Purchased Data Rate, dtype: int64

In [114]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# summary_df['Due Date'].value_counts()

In [115]:
from datetime import timedelta, date
from pandas.tseries.offsets import DateOffset

yesterday_date = date.today() - timedelta(days=90)
future_date = date.today() + timedelta(days=90)
# wire center level - if new speeds pop up, add to below list to create new columns
#combine2_sum_df = combine2_df.groupby(by = ['WIRE_CENTER_ID', 'Purchased Data Rate', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Work Order Number'].nunique()

# state level - get total purchases
# combine2_state_df.groupby(by = ['STATE_CD', 'Purchased Data Rate'], as_index = False)['Work Order Number'].nunique()
speed_940m = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_940m = speed_940m.rename(columns = {'Order Number': '940M - Active Subscribers'})

# speed_940m_pending = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_940m_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                                &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                                &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_940m_pending = speed_940m_pending.rename(columns = {'Order Number': '940M - Pending Activation Subscribers'})

# 200M

speed_200m = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '200M/200M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_200m = speed_200m.rename(columns = {'Order Number': '200M - Active Subscribers'})

speed_200m_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                                &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                                &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '200M/200M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_200m_pending = speed_200m_pending.rename(columns = {'Order Number': '200M - Pending Activation Subscribers'})

# 1G

speed_1g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '1G/1G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_1g = speed_1g.rename(columns = {'Order Number': '1G - Active Subscribers'})

speed_1g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '1G/1G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_1g_pending = speed_1g_pending.rename(columns = {'Order Number': '1G - Pending Activation Subscribers'})

# 3G
speed_3g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '3G/3G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_3g = speed_3g.rename(columns = {'Order Number': '3G - Active Subscribers'})

speed_3g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '3G/3G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_3g_pending = speed_3g_pending.rename(columns = {'Order Number': '3G - Pending Activation Subscribers'})

# 8G

speed_8g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '8G/8G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_8g = speed_8g.rename(columns = {'Order Number': '8G - Active Subscribers'})

speed_8g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '8G/8G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_8g_pending = speed_8g_pending.rename(columns = {'Order Number': '8G - Pending Activation Subscribers'})

In [116]:
final_summary = speed_8g_pending.merge(speed_8g.merge(speed_3g_pending.merge((speed_3g.merge((speed_1g_pending.merge((speed_1g.merge((speed_200m_pending.merge((speed_200m.merge((speed_940m.merge(speed_940m_pending, how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])


In [117]:
final_summary = final_summary.fillna(0)

In [118]:
# final_summary['Aggregate 200M Penetration Rate'] = final_summary['200M - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 940M Penetration Rate'] = final_summary['940M - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 1G Penetration Rate'] = final_summary['1G - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 3G Penetration Rate'] = final_summary['3G - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']

In [119]:
final_summary['Total Active Subscribers'] = final_summary['8G - Active Subscribers'] + final_summary['3G - Active Subscribers'] + final_summary['1G - Active Subscribers'] + final_summary['200M - Active Subscribers'] + final_summary['940M - Active Subscribers']

In [120]:
final_summary['Total Pending Activation Subscribers'] = final_summary['8G - Pending Activation Subscribers'] + final_summary['3G - Pending Activation Subscribers'] + final_summary['1G - Pending Activation Subscribers'] + final_summary['200M - Pending Activation Subscribers'] + final_summary['940M - Pending Activation Subscribers']

In [121]:
final_summary['Aggregate Penetration Rate (Active Subs)'] = final_summary['Total Active Subscribers'] / final_summary['Enabled_LUs_WireCenter_Level']

In [124]:
# uncomment to check data
# final_summary[final_summary['Total Pending Activation Subscribers']>0]

In [125]:
# join wire center level mg enabled lu count

final_summary_mg = final_summary.merge(mg_lu_df, on = 'WIRE_CENTER_ID', how = 'left')
final_summary_mg['MG_ENABLED_LU_CNT'] = final_summary_mg['MG_ENABLED_LU_CNT'].fillna(0)
final_summary_mg = final_summary_mg.rename(columns = {'MG_ENABLED_LU_CNT': 'Multigig_Enabled_LUs'})

In [126]:
state_summary = final_summary_mg.groupby(['STATE'], as_index = False).sum()

In [127]:
state_summary = state_summary[['STATE','Enabled_LUs_WireCenter_Level','Multigig_Enabled_LUs','8G - Pending Activation Subscribers','8G - Active Subscribers','3G - Pending Activation Subscribers','3G - Active Subscribers','1G - Pending Activation Subscribers','1G - Active Subscribers','940M - Active Subscribers','940M - Pending Activation Subscribers','200M - Pending Activation Subscribers','200M - Active Subscribers','Total Active Subscribers','Total Pending Activation Subscribers']]

In [128]:
# add state_summary aggregate penetration rate

state_summary['Aggregate Penetration Rate (Active Subs)'] = state_summary['Total Active Subscribers']/state_summary['Enabled_LUs_WireCenter_Level']

In [129]:
state_summary

,STATE,Enabled_LUs_WireCenter_Level,Multigig_Enabled_LUs,8G - Pending Activation Subscribers,8G - Active Subscribers,3G - Pending Activation Subscribers,3G - Active Subscribers,1G - Pending Activation Subscribers,1G - Active Subscribers,940M - Active Subscribers,940M - Pending Activation Subscribers,200M - Pending Activation Subscribers,200M - Active Subscribers,Total Active Subscribers,Total Pending Activation Subscribers,Aggregate Penetration Rate (Active Subs)
0,0,0.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,39,16,0.0,0.0,57.0,NaN
1,AZ,11355.0,9101.0,0,0.0,0.0,1.0,1,10.0,93.0,18,2,39.0,143.0,21.0,0.012594
2,CO,7267.0,1211.0,0,0.0,0.0,1.0,2,27.0,45.0,14,8,28.0,101.0,24.0,0.013898
3,FL,19705.0,0.0,0,0.0,0.0,0.0,0,0.0,84.0,226,86,13.0,97.0,312.0,0.004923
4,MN,8362.0,8362.0,0,0.0,0.0,0.0,11,14.0,24.0,9,2,32.0,70.0,22.0,0.008371
5,OR,264.0,264.0,0,0.0,0.0,1.0,0,0.0,0.0,0,0,0.0,1.0,0.0,0.003788
6,UT,2155.0,170.0,0,0.0,0.0,0.0,0,0.0,12.0,12,5,6.0,18.0,17.0,0.008353
7,WA,2286.0,340.0,0,0.0,0.0,2.0,0,11.0,40.0,35,1,3.0,56.0,36.0,0.024497


In [130]:
# create MG state summary
# 10/12/22 - comment below out to allow null states to be counted
state_summary_mg = state_summary[(state_summary['Multigig_Enabled_LUs']>0) | (state_summary['STATE'] == 0)]

In [131]:
state_summary_mg = state_summary_mg.drop(['Aggregate Penetration Rate (Active Subs)','Enabled_LUs_WireCenter_Level','200M - Pending Activation Subscribers','200M - Active Subscribers','940M - Active Subscribers','940M - Pending Activation Subscribers'], axis = 1)

In [132]:
state_summary_mg['Total Active Subscribers'] = state_summary_mg['8G - Active Subscribers'] + state_summary_mg['3G - Active Subscribers'] + state_summary_mg['1G - Active Subscribers']

In [133]:
state_summary_mg['Total Pending Activation Subscribers'] = state_summary_mg['8G - Pending Activation Subscribers'] + state_summary_mg['3G - Pending Activation Subscribers'] + state_summary_mg['1G - Pending Activation Subscribers'] 

In [134]:
state_summary_mg['MG Penetration Rate (Active Subs)'] = state_summary_mg['Total Active Subscribers'] / state_summary_mg['Multigig_Enabled_LUs']

In [135]:
# reorder columns
columns1 = ['STATE', 'Multigig_Enabled_LUs', 'Total Active Subscribers',
       'Total Pending Activation Subscribers',
       'MG Penetration Rate (Active Subs)',
       '8G - Active Subscribers', '8G - Pending Activation Subscribers',
       '3G - Active Subscribers','3G - Pending Activation Subscribers', 
       '1G - Active Subscribers','1G - Pending Activation Subscribers']

state_summary_mg = state_summary_mg[columns1]

columns2 = ['STATE', 'Enabled_LUs_WireCenter_Level', 'Multigig_Enabled_LUs',
        'Total Active Subscribers', 'Total Pending Activation Subscribers',
       'Aggregate Penetration Rate (Active Subs)',
       '8G - Active Subscribers','8G - Pending Activation Subscribers', 
       '3G - Active Subscribers','3G - Pending Activation Subscribers', 
       '1G - Active Subscribers','1G - Pending Activation Subscribers', 
       '940M - Active Subscribers', '940M - Pending Activation Subscribers',
        '200M - Active Subscribers','200M - Pending Activation Subscribers']

state_summary = state_summary[columns2]

columns3 = ['STATE', 'Enabled_LUs_WireCenter_Level', 'Multigig_Enabled_LUs',
            '200M - Active Subscribers',
            '940M - Active Subscribers', 
            '1G - Active Subscribers',
            '3G - Active Subscribers',
            '8G - Active Subscribers',
            'Total Active Subscribers'
]

state_summary_view = state_summary[columns3]

In [136]:
# ready for export
state_summary_view = state_summary_view[state_summary_view['STATE'] != 0].sort_values(by = 'Enabled_LUs_WireCenter_Level', ascending = False)

In [137]:
import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_Salesforce_Summary_StateWCLevel.xlsx"

with pd.ExcelWriter(excelfilename) as writer:  
    state_summary.to_excel(writer, sheet_name='State_Summary', index = False)
    state_summary_mg.to_excel(writer, sheet_name='MultiGig_State_Summary', index = False)
    final_summary.to_excel(writer, sheet_name='Summary', index = False)
    sf_df.to_excel(writer, sheet_name='Details', index = False)




### Fill template

In [138]:
# check number of rows = 7 otherwise move grand total position in templates:
state_summary_view

,STATE,Enabled_LUs_WireCenter_Level,Multigig_Enabled_LUs,200M - Active Subscribers,940M - Active Subscribers,1G - Active Subscribers,3G - Active Subscribers,8G - Active Subscribers,Total Active Subscribers
3,FL,19705.0,0.0,13.0,84.0,0.0,0.0,0.0,97.0
1,AZ,11355.0,9101.0,39.0,93.0,10.0,1.0,0.0,143.0
4,MN,8362.0,8362.0,32.0,24.0,14.0,0.0,0.0,70.0
2,CO,7267.0,1211.0,28.0,45.0,27.0,1.0,0.0,101.0
7,WA,2286.0,340.0,3.0,40.0,11.0,2.0,0.0,56.0
6,UT,2155.0,170.0,6.0,12.0,0.0,0.0,0.0,18.0
5,OR,264.0,264.0,0.0,0.0,0.0,1.0,0.0,1.0


In [139]:
with pd.ExcelWriter('View/Multigig_Sales_View.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    state_summary_view.to_excel(writer, 
                      sheet_name="Sheet1", 
                      startrow=2, 
                      index=False, 
                      header=False) 

In [140]:
# create archived copy
import shutil

excelfilename = "View/archive/" + TodaysDate +"_Multigig_Sales_View.xlsx"
shutil.copyfile('View/Multigig_Sales_View.xlsx', excelfilename)

'View/archive/111722_Multigig_Sales_View.xlsx'